In [1]:
import os
import jax
from jax import random
import jax.numpy as jnp
import numpy as np
from meta_transformer import torch_utils, module_path, on_cluster
import flax.linen as nn
from jax import random
import nnaugment
from nnaugment.conventions import import_params, export_params
rng = random.PRNGKey(42)

import torch

/home/lauro/.virtualenvs/meta-models/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load pytorch nets

In [2]:
if not on_cluster:
    # dpath = os.path.join(module_path, "data/david_backdoors")  # local
    # use for testing with small dataset sizes (only works if rds storage is mounted):
    dpath = os.path.join(module_path, "/home/lauro/rds/model-zoo/")
else:
    dpath = "/rds/user/lsl38/rds-dsk-lab-eWkDxBhxBrQ/model-zoo/"  

model_dataset_paths = {
    "mnist": "mnist-cnns",
    "cifar10": "cifar10",
    "svhn": "svhn",
}


model_dataset_paths = {
    k: os.path.join(dpath, v) for k, v in model_dataset_paths.items()
}


inputs, targets, get_pytorch_model = torch_utils.load_input_and_target_weights(
    model=torch_utils.CNNMedium(),
    num_models=10,
    data_dir=model_dataset_paths["cifar10"],
    inputs_dirname="poison_noL1",
    targets_dirname="clean",
)

param_list = [import_params(p, from_naming_scheme="pytorch") for p in inputs]
params = param_list[0]

/home/lauro/.virtualenvs/meta-models/lib/python3.8/site-packages/einops/parsing.py:136: RuntimeWarning: It is discouraged to use axes names that are keywords: in
  warnings.warn("It is discouraged to use axes names that are keywords: {}".format(name), RuntimeWarning)


In [6]:
augmented_params = nnaugment.random_permutation(
    params,
    layers_to_permute=[f'Conv2d_{i}' for i in range(6)] + ['Dense_6'],
)

In [15]:
model = get_pytorch_model(export_params(params))
model.eval()
perm_model = get_pytorch_model(export_params(augmented_params))
perm_model.eval()

batch_size = 5
input_tensor = torch.randn(batch_size, 3, 32, 32)

# Feed the tensor to the model
with torch.no_grad():
    output = model(input_tensor)
    perm_output = perm_model(input_tensor)
    assert torch.allclose(output, perm_output, atol=1e-6, rtol=1e-3), "Outputs differ after weight augmentation."

# Test stuff

In [15]:
np.set_printoptions(precision=2, suppress=True)


def print_compact_4d(arr):
    # Convert array to string
    arr_str = str(arr)
    
    # Replace multiple line breaks
    compact_str = arr_str.replace("\n\n\n", "\n---\n")
#    compact_str = compact_str.replace("\n\n", "\n")
    
    # Print the compact string
    print(compact_str)


def print_params(variables):
    for name, layer in variables['params'].items():
        print(name)
        print("w:")
        print_compact_4d(layer["kernel"])
        print()
        print("b:")
        print(layer["bias"])
        print("\n====\n")

In [16]:
arr = np.array([3,1,2])
arr = np.concatenate([arr + len(arr) * i for i in range(3)])
arr

array([3, 1, 2, 6, 4, 5, 9, 7, 8])

In [17]:
print_params(variables)

print()
print("Compare to augmented params:")
print_params(augmented_variables)

NameError: name 'variables' is not defined

In [ ]:
initial_output

Array([-0.03], dtype=float32)

In [ ]:
augmented_output

Array([-0.03], dtype=float32)

In [ ]:
np.abs(initial_output - augmented_output)

array([0.], dtype=float32)